# Build a Simple LLM Application with LCEL

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After reading this tutorial, you'll have a high level overview of:

- Using [language models](/docs/concepts/chat_models)

- Using [PromptTemplates](/docs/concepts/prompt_templates)

- Debugging and tracing your application using [LangSmith](https://docs.smith.langchain.com/)

- Deploying your application with [LangGraph Platform](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/)

Let's dive in!

## Setup

### Jupyter Notebook

This and other tutorials are perhaps most conveniently run in a [Jupyter notebooks](https://jupyter.org/). Going through guides in an interactive environment is a great way to better understand them. See [here](https://jupyter.org/install) for instructions on how to install.

### Installation

To install LangChain run:

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install langchain</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```

## Using Language Models

First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangeably. For details on getting started with a specific model, refer to [supported integrations](/docs/integrations/chat/).

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4o-mini"`} />


In [1]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

Let's first use the model directly. [ChatModels](/docs/concepts/chat_models) are instances of LangChain [Runnables](/docs/concepts/runnables/), which means they expose a standard interface for interacting with them. To simply call the model, we can pass in a list of messages to the `.invoke` method.

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_9ee9e968ea', 'finish_reason': 'stop', 'logprobs': None}, id='run-ad371806-6082-45c3-b6fa-e44622848ab2-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

If we've enabled LangSmith, we can see that this run is logged to LangSmith, and can see the [LangSmith trace](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r). The LangSmith trace reports [token](/docs/concepts/tokens/) usage information, latency, [standard model parameters](/docs/concepts/chat_models/#standard-parameters) (such as temperature), and other information.

Note that ChatModels receive [message](/docs/concepts/messages/) objects as input and generate message objects as output. In addition to text content, message objects convey conversational [roles](/docs/concepts/messages/#role) and hold important data, such as [tool calls](/docs/concepts/tool_calling/) and token usage counts.

## Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

[Prompt templates](/docs/concepts/prompt_templates/) are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model. 

Let's create a `PromptTemplate` here. It will take in two user variables:

- `language`: The language to translate text into
- `text`: The text to translate

In [3]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

Note that `ChatPromptTemplate` supports multiple [message roles](/docs/concepts/messages/#role) in a single template. We format the `language` parameter into the system message, and the user `text` into a user message.

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [4]:
result = prompt_template.invoke({"language": "Italian", "text": "hi!"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

We can see that it returns a `ChatPromptValue` that consists of two messages. If we want to access the messages directly we do:

In [5]:
result.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})]

## Chaining together components with LCEL

We can now combine this with the model from above using the pipe (`|`) operator:

In [6]:
chain = prompt_template | model

In [7]:
response = chain.invoke({"language": "Italian", "text": "hi!"})
print(response.content)

Ciao!


This is a simple example of using [LangChain Expression Language (LCEL)](/docs/concepts/lcel) to chain together LangChain modules. There are several benefits to this approach, including optimized streaming and tracing support.

If we take a look at the [LangSmith trace](https://smith.langchain.com/public/bc49bec0-6b13-4726-967f-dbd3448b786d/r), we can see both components show up.

## Serving with LangGraph Platform

Now that we've built an application, we need to serve it. That's where [LangGraph Platform](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/) comes in. LangGraph Platform is designed to make deploying LLM applications seamless and production-ready. You do not need to use LangGraph Platform to use LangChain, but in this guide we'll show how you can use it to deploy your application. We won't go into detail on LangGraph or LangGraph Platform concepts here; consult its [documentation](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/) for detail on its features and in-depth guides.

While the first part of this guide was intended to be run in a Jupyter Notebook or script, we will now move out of that. We will use command-line tools to create a set of files and deploy them locally.

1. __Create application directory and files__

    We will use the [langgraph-cli](https://langchain-ai.github.io/langgraph/concepts/langgraph_cli/) to initialize a template application. This will give us a working scaffold of an application that we can customize. You can install `langgraph-cli` via Homebrew (on macOS) or pip. See its [documentation](https://langchain-ai.github.io/langgraph/concepts/langgraph_cli/) for detail.

    ```shell
    pip install "langgraph-cli>=0.1.53"
    ```

    Let's create a folder to house our application:

    ```shell
    mkdir my-app && cd my-app
    ```

    We can now use the command `langgraph new` to download a [template application](https://langchain-ai.github.io/langgraph/concepts/template_applications/). This command asks you to specify:
   - The path -- we can select the current directory with `.`
   - The template -- enter `1` to select the "New LangGraph Project" template
   - Python or JS -- enter `1` to select Python
  
    With that, we've created a functional, minimal application that we can customize. There are four files that we will need to change to implement our translator bot.

1. __Define your applicaton logic__

    We will need to update the application to run our translator bot. This will touch two files. First, we update the application's [state](https://langchain-ai.github.io/langgraph/concepts/low_level/#state) to include fields for the language, input text, and response:

   In `src/agent/state.py`:

   ```python
    @dataclass
    class State:
        """Defines the input state for the agent, representing a narrower interface to the outside world.
    
        This class is used to define the initial state and structure of incoming data.
        See: https://langchain-ai.github.io/langgraph/concepts/low_level/#state
        for more information.
        """
        # highlight-start
        text: str
        language: str
        response: str = ""
        # highlight-end
   ```

   Next, we will update the application logic to run our translator bot.

   In `src/agent/graph.py`:

   ```python
    from typing import Any, Dict

    # highlight-next-line
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.runnables import RunnableConfig
    # highlight-next-line
    from langchain_openai import ChatOpenAI
    from langgraph.graph import StateGraph
    
    from agent.configuration import Configuration
    from agent.state import State
    

    # highlight-start
    model = ChatOpenAI(model="gpt-4o")
    
    system_template = "Translate the following from English into {language}"
    
    prompt_template = ChatPromptTemplate.from_messages(
        [("system", system_template), ("user", "{text}")]
    )
    
    chain = prompt_template | model
    # highlight-end
    
    
    async def my_node(state: State, config: RunnableConfig) -> Dict[str, Any]:
        """Each node does work."""
        # highlight-start
        response = chain.invoke({"language": state.language, "text": state.text})
        return {"response": response.content}
        # highlight-end
    
    
    # Define a new graph
    workflow = StateGraph(State, config_schema=Configuration)
    
    # Add the node to the graph
    workflow.add_node("my_node", my_node)
    
    # Set the entrypoint as `call_model`
    workflow.add_edge("__start__", "my_node")
    
    # Compile the workflow into an executable graph
    graph = workflow.compile()
    graph.name = "New Graph"  # This defines the custom name in LangSmith

   ```

1. __Specify environment variables__

   We should add our OpenAI API keys and other necessary settings or secrets to a `.env` file. Populate the API key in the `.env.example` file and re-name it to `.env`.

1. __Specify dependencies__

   Finally, we need to add `langchain-openai` or any other required packages to `pyproject.toml`:

   ```
    dependencies = [
        "langgraph>=0.2.6",
        "python-dotenv>=1.0.1",
        "langchain-openai>=0.2",
    ]
   ```

1. __Deploy__

    We can again use `langgraph-cli` to deploy locally:

   ```shell
   langgraph up
   ```

   This will build the application with Docker and run it behind `localhost` on your machine.

1. __Test__

    We can use the `ok` endpoint to test that the application is ready:

   ```shell
   curl --request GET --url http://localhost:8123/ok
   ```

   If you receive `{"ok":true}`, the application is ready for use.


LangGraph Platform supports several means of interacting with the server. Below we will demonstrate the [SDK](https://langchain-ai.github.io/langgraph/concepts/sdk/).

### Client

Install with:
```bash
pip install --upgrade langgraph-sdk
```

Assuming our deployment is at `http://localhost:8123`, below we initialize a client and identify the corresponding [Assistant](https://langchain-ai.github.io/langgraph/concepts/assistants/). An Assistant is an instance of our application.

```python
from langgraph_sdk import get_client

client = get_client(url="http://localhost:8123")
# get default assistant
assistants = await client.assistants.search(metadata={"created_by": "system"})
assistant = assistants[0]
```

We can now query the Assistant in a ([stateless](https://langchain-ai.github.io/langgraph/cloud/how-tos/stateless_runs/)) run:

```python
input = {"language": "Italian", "text": "hi!"}

async for chunk in client.runs.stream(
    None,  # stateless run
    assistant["assistant_id"],
    input=input,
    stream_mode="updates",
):
    if chunk.data:
        print(chunk.data)
```

```
{'run_id': '1efa1faa-23ce-69cb-858a-97f292fc5561', 'attempt': 1}
{'my_node': {'response': 'Ciao!'}}
```

### Studio UI

Every LangGraph Platform comes with a simple [built-in UI](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/) for configuring and invoking the application with streaming output and visibility into intermediate steps.
Head to [https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:8123](https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:8123) to try it out! Pass in the same inputs as before - `{"language": "italian", "text": "hi"}` - and it should respond same as before.

## Conclusion

That's it! In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to how to create a prompt template, how to get great observability into chains you create with LangSmith, and how to deploy them with LangGraph Platform.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!

For further reading on the core concepts of LangChain, we've got detailed [Conceptual Guides](/docs/concepts).

If you have more specific questions on these concepts, check out the following sections of the how-to guides:

- [Chat models](/docs/how_to/#chat-models)
- [Prompt templates](/docs/how_to/#prompt-templates)

As well as the docs for [LangSmith](https://docs.smith.langchain.com) and [LangGraph Platform](https://langchain-ai.github.io/langgraph/concepts/langgraph_platform/).